<h1>World Population - Web Scraping and Visualization<h1>
    <hr>

<h2>1) Import Libraries<h2>

In [ ]:
#!pip install gspread
import gspread
import numpy as np
import pandas as pd
import requests
import bs4
import lxml.etree as xml
import seaborn as sns
import matplotlib.pyplot as plt 
from datetime import timedelta
# if using a Jupyter notebook, include:
%matplotlib inline
import sys
#!{sys.executable} -m pip install selenium

<h2>2) URL Text Extraction<h2> 

In [18]:
# Request HTML document
url = "https://www.worldometers.info/world-population/population-by-country/"
requests.get(url)
requests.get(url, {}).text
soup = bs4.BeautifulSoup(requests.get(url,{}).text, "lxml")
soup.head.title

# Retrieve text from URL
r = requests.get(url)
html_doc = r.text

# Parse HTML table
soup = bs4.BeautifulSoup(html_doc, features='html.parser')

table = soup.find('table', attrs={'id':'example2'})

list1 = []

for tr in table.find_all('tr'):
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    list1.append(row)

<title>Population by Country (2022) - Worldometer</title>

<h2>3) Data Pre-processing<h2>

In [19]:
# Create DataFrame       
df=pd.DataFrame(list1)  

# Data Transformation
df.columns =  df.iloc[1]

    # Find Missing Value
    # df['abc'].isnull().sum() 

    # Deduplication
    # df['abc'].drop_duplicates() 
    
    # Standardization
    # df['abc']= pd.to_datetime(df["abc"])
    # Convert integer to date format
    
    # Invalid data - Non-negative constarint, Valid date
    # sum((pd.to_datetime(df['date_published']) >= "1/1/1906") & (pd.to_datetime(df['date_published']) <= "12/31/2020"))
    # sum(df['abc']<0)
    # Result: abc


# Starting from this point,it's just how you want to clean and slice the data
df = df.iloc[1:]  #check the data to see if you want to only read these
df.dropna(axis='columns', how='all', inplace=True)
df.drop(df.columns[[0]], axis = 1, inplace=True) 
df.columns = ['Country (or dependency)', 'Population (2020)', 'Yearly Change', 'Net Change', 
                'Density (P/Km^2)', 'Land Area (Km^2)', 'Migrants (net)', 'Fert. Rate', 'Med. Age', 'Urban Pop%', 'World Share'] 
df.replace(',','', regex=True, inplace=True)
df['Population (2020)'].apply(pd.to_numeric,errors='coerce')
df['Net Change'].apply(pd.to_numeric,errors='coerce')
df['Density (P/Km^2)'].apply(pd.to_numeric,errors='coerce')
df['Land Area (Km^2)'].apply(pd.to_numeric,errors='coerce')
df['Migrants (net)'].apply(pd.to_numeric,errors='coerce')
df['Fert. Rate'].apply(pd.to_numeric,errors='coerce')
df['Med. Age'].apply(pd.to_numeric,errors='coerce')
df

,Country (or dependency),Population (2020),Yearly Change,Net Change,Density (P/Km^2),Land Area (Km^2),Migrants (net),Fert. Rate,Med. Age,Urban Pop%,World Share
1,China,1439323776,0.39 %,5540090,153,9388211,-348399,1.7,38,61 %,18.47 %
2,India,1380004385,0.99 %,13586631,464,2973190,-532687,2.2,28,35 %,17.70 %
3,United States,331002651,0.59 %,1937734,36,9147420,954806,1.8,38,83 %,4.25 %
4,Indonesia,273523615,1.07 %,2898047,151,1811570,-98955,2.3,30,56 %,3.51 %
5,Pakistan,220892340,2.00 %,4327022,287,770880,-233379,3.6,23,35 %,2.83 %
...,...,...,...,...,...,...,...,...,...,...,...
231,Montserrat,4992,0.06 %,3,50,100,,N.A.,N.A.,10 %,0.00 %
232,Falkland Islands,3480,3.05 %,103,0,12170,,N.A.,N.A.,66 %,0.00 %
233,Niue,1626,0.68 %,11,6,260,,N.A.,N.A.,46 %,0.00 %
234,Tokelau,1357,1.27 %,17,136,10,,N.A.,N.A.,0 %,0.00 %


<h2>4) Load Dataframe to Google Sheet<h2>

In [23]:
# Open GG Sheet file using Credential Key
gc = gspread.service_account(filename='ggsheet-344513-d147337e1b01.json')
sh = gc.open("Population_by_Country")

# Add Dataset Worksheet
worksheet = sh.add_worksheet(title="Dataset", rows=df.shape[0], cols=df.shape[1])

# Add DataFrame to worksheet
worksheet.update([df.columns.values.tolist()]+df.values.tolist())

# Add new row
# worksheet.append_row(df.loc[0].values.tolist())

{'spreadsheetId': '1J45ZZpJtmy1ItjT9BF3zxS2wF-g9pCWlmOdvS9UApsY',
 'updatedRange': 'Dataset!A1:K236',
 'updatedRows': 236,
 'updatedColumns': 11,
 'updatedCells': 2596}